In [1]:
library(lme4)
library(MASS)
library(plyr) 
library(knitr)
library(multcomp)
library(FSA)
library(car)
library(multcomp)
# library(PMCMRplus)
# library(PMCMR)
# library(rcompanion)
# library(glmmTMB)

Loading required package: Matrix

Loading required package: mvtnorm

Loading required package: survival

Loading required package: TH.data


Attaching package: 'TH.data'


The following object is masked from 'package:MASS':

    geyser


## FSA v0.9.4. See citation('FSA') if used in publication.
## Run fishR() for related website and fishR('IFAR') for related book.


Attaching package: 'FSA'


The following object is masked from 'package:plyr':

    mapvalues


Loading required package: carData

Registered S3 methods overwritten by 'car':
  method       from
  hist.boot    FSA 
  confint.boot FSA 


Attaching package: 'car'


The following object is masked from 'package:FSA':

    bootCase




In [2]:
file_path = 'combined_data_5_20_data_analysis.csv'
data <- read.csv(file_path)

In [3]:
print.model.summary <- function(model){
    print('***********Coefficients of model*******************')
    print(summary(model)$coef)
    print('***************************************************')
}

In [4]:
print.anova_stats <- function(model, data, p_val){
    stats = anova(model)
    df_total <- length(data$Id)
    df_vec <- stats$npar
    print(stats)
    f_vec <- stats$"F value"
    for (i in 1:length(df_vec)) {
      p <- pf(f_vec[i], df_vec[i], df_total-df_vec[i], lower.tail = FALSE, log.p = FALSE)
      if(p < p_val){
        (message(rownames(stats)[i],":  F(",df_vec[i],",",df_total,")=",f_vec[i],"     p = ",p))
          }

    }
}

In [5]:
#+++++++++++++++++++++++++
# Function to calculate the mean and the standard deviation
  # for each group
#+++++++++++++++++++++++++
# data : a data frame
# varname : the name of a column containing the variable
  #to be summariezed
# groupnames : vector of column names to be used as
  # grouping variables
data_summary <- function(data, varname, groupnames){
  require(plyr)
  summary_func <- function(x, col){
    c(mean = mean(x[[col]], na.rm=TRUE),
      sd = sd(x[[col]], na.rm=TRUE)/sqrt(length(data)))
  }
  data_sum<-ddply(data, groupnames, .fun=summary_func,
                  varname)
  data_sum <- rename(data_sum, c("mean" = varname))
 return(data_sum)
}

## Performance

### 1. Score or Total Reward

In [6]:
# rANOVA fails test assuptions
m <- lmer(score ~ factor(data$condition_n_explanation) + data$round 
          + factor(data$gender) + data$age + data$experience
          + data$extraversion + data$agreeableness + data$conscientiousness + data$neuroticism + data$intellect 
          + (1|data$Id), data)

# # Test for model assumptions
shapiro.test(residuals(m))  # p value should be greater than 0.05 to satisfy normality assumptions
# leveneTest(data$trust ~ factor(data$condition_n_explanation) * factor(data$round))  # p value should be greater than 0.05 to satisfy homoscedasticity assumptions (only use categorical variables)

# print(Anova(m))
print.anova_stats(m, data, 0.05)


	Shapiro-Wilk normality test

data:  residuals(m)
W = 0.97211, p-value = 2.016e-05


Analysis of Variance Table
                                     npar Sum Sq Mean Sq F value
factor(data$condition_n_explanation)    4 101035 25258.8 19.7664
data$round                              1    362   362.0  0.2833
factor(data$gender)                     1   2036  2036.4  1.5936
data$age                                1    550   550.4  0.4307
data$experience                         1     13    12.7  0.0099
data$extraversion                       1   6546  6545.7  5.1223
data$agreeableness                      1    293   292.6  0.2290
data$conscientiousness                  1   3306  3305.6  2.5868
data$neuroticism                        1   3326  3325.5  2.6024
data$intellect                          1   1674  1674.2  1.3102


factor(data$condition_n_explanation):  F(4,290)=19.7663504411515     p = 2.21865136535073e-14

data$extraversion:  F(1,290)=5.12233605425222     p = 0.0243615583089973



In [7]:
kruskal.test(data$score ~ factor(data$condition_n_explanation)) 
kruskal.test(data$score ~ data$round)
kruskal.test(data$score ~ data$condition_n_round)


	Kruskal-Wallis rank sum test

data:  data$score by factor(data$condition_n_explanation)
Kruskal-Wallis chi-squared = 58.159, df = 4, p-value = 7.066e-12



	Kruskal-Wallis rank sum test

data:  data$score by data$round
Kruskal-Wallis chi-squared = 0.059397, df = 1, p-value = 0.8075



	Kruskal-Wallis rank sum test

data:  data$score by data$condition_n_round
Kruskal-Wallis chi-squared = 61.52, df = 9, p-value = 6.824e-10


In [8]:
dunnTest(data$score, factor(data$condition_n_explanation))

Dunn (1964) Kruskal-Wallis multiple comparison

  p-values adjusted with the Holm method.




                                Comparison          Z      P.unadj        P.adj
1              control - control_w_explain -0.6261877 5.311918e-01 5.311918e-01
2                      control - interrupt  2.7616483 5.751040e-03 2.875520e-02
3            control_w_explain - interrupt  3.3878359 7.044639e-04 4.931247e-03
4            control - interrupt_w_explain  0.6394755 5.225137e-01 1.000000e+00
5  control_w_explain - interrupt_w_explain  1.2656632 2.056337e-01 6.169011e-01
6          interrupt - interrupt_w_explain -2.1221728 3.382323e-02 1.352929e-01
7                   control - no_interrupt  6.0060867 1.900546e-09 1.710492e-08
8         control_w_explain - no_interrupt  6.6322744 3.305534e-11 3.305534e-10
9                 interrupt - no_interrupt  3.2444384 1.176825e-03 7.060948e-03
10      interrupt_w_explain - no_interrupt  5.3666112 8.022975e-08 6.418380e-07

In [9]:
Hadj <- unname(kruskal.test(data$score ~ factor(data$condition_n_explanation))$statistic)
n <- sum(table(data$score, factor(data$condition_n_explanation)))
e2 <- Hadj*(n+1)/(n^2-1)
print(Hadj)  # H-value (test statistic => chi-squared)
print(e2)  # Effect Size

[1] 58.15909
[1] 0.2012425


### 2. Extra Steps

In [10]:
# rANOVA fails test assuptions
m <- lmer(extra_step ~ factor(data$condition_n_explanation) + data$round 
          + factor(data$gender) + data$age + data$experience
          + data$extraversion + data$agreeableness + data$conscientiousness + data$neuroticism + data$intellect 
          + (1|data$Id), data)

# # Test for model assumptions
shapiro.test(residuals(m))  # p value should be greater than 0.05 to satisfy normality assumptions
# leveneTest(data$trust ~ factor(data$condition_n_explanation) * factor(data$round))  # p value should be greater than 0.05 to satisfy homoscedasticity assumptions (only use categorical variables)

# print(Anova(m))
print.anova_stats(m, data, 0.05)


	Shapiro-Wilk normality test

data:  residuals(m)
W = 0.97548, p-value = 7.126e-05


Analysis of Variance Table
                                     npar Sum Sq Mean Sq F value
factor(data$condition_n_explanation)    4 33.317  8.3292 14.6585
data$round                              1  1.410  1.4103  2.4820
factor(data$gender)                     1  0.187  0.1866  0.3285
data$age                                1  0.443  0.4430  0.7797
data$experience                         1  0.063  0.0631  0.1111
data$extraversion                       1  0.803  0.8031  1.4133
data$agreeableness                      1  0.628  0.6277  1.1046
data$conscientiousness                  1  1.169  1.1689  2.0571
data$neuroticism                        1  0.806  0.8062  1.4188
data$intellect                          1  1.199  1.1993  2.1107


factor(data$condition_n_explanation):  F(4,290)=14.6585212837656     p = 6.6334236632083e-11



In [11]:
kruskal.test(data$extra_step ~ factor(data$condition_n_explanation)) 
kruskal.test(data$extra_step ~ data$round)
kruskal.test(data$extra_step ~ data$condition_n_round)


	Kruskal-Wallis rank sum test

data:  data$extra_step by factor(data$condition_n_explanation)
Kruskal-Wallis chi-squared = 52.712, df = 4, p-value = 9.791e-11



	Kruskal-Wallis rank sum test

data:  data$extra_step by data$round
Kruskal-Wallis chi-squared = 1.3531, df = 1, p-value = 0.2447



	Kruskal-Wallis rank sum test

data:  data$extra_step by data$condition_n_round
Kruskal-Wallis chi-squared = 56.937, df = 9, p-value = 5.197e-09


In [12]:
dunnTest(data$extra_step, factor(data$condition_n_explanation))

Dunn (1964) Kruskal-Wallis multiple comparison

  p-values adjusted with the Holm method.




                                Comparison          Z      P.unadj        P.adj
1              control - control_w_explain  1.0308837 3.025954e-01 6.051907e-01
2                      control - interrupt -4.3732280 1.224228e-05 8.569594e-05
3            control_w_explain - interrupt -5.4041117 6.513036e-08 5.861732e-07
4            control - interrupt_w_explain -2.3845414 1.710042e-02 8.550211e-02
5  control_w_explain - interrupt_w_explain -3.4154252 6.368253e-04 3.820952e-03
6          interrupt - interrupt_w_explain  1.9886865 4.673581e-02 1.402074e-01
7                   control - no_interrupt -4.7258987 2.290998e-06 1.832799e-05
8         control_w_explain - no_interrupt -5.7567824 8.573229e-09 8.573229e-08
9                 interrupt - no_interrupt -0.3526707 7.243353e-01 7.243353e-01
10      interrupt_w_explain - no_interrupt -2.3413573 1.921377e-02 7.685508e-02

### 3. Fallling Number

In [13]:
kruskal.test(data$falling_num ~ factor(data$condition_n_explanation)) 
kruskal.test(data$falling_num ~ data$round)
kruskal.test(data$falling_num ~ data$condition_n_round)


	Kruskal-Wallis rank sum test

data:  data$falling_num by factor(data$condition_n_explanation)
Kruskal-Wallis chi-squared = 76.961, df = 4, p-value = 7.665e-16



	Kruskal-Wallis rank sum test

data:  data$falling_num by data$round
Kruskal-Wallis chi-squared = 0.33692, df = 1, p-value = 0.5616



	Kruskal-Wallis rank sum test

data:  data$falling_num by data$condition_n_round
Kruskal-Wallis chi-squared = 80.273, df = 9, p-value = 1.427e-13


In [14]:
dunnTest(data$falling_num, factor(data$condition_n_explanation))

Dunn (1964) Kruskal-Wallis multiple comparison

  p-values adjusted with the Holm method.




                                Comparison          Z      P.unadj        P.adj
1              control - control_w_explain  0.2577293 7.966158e-01 7.966158e-01
2                      control - interrupt -3.3841972 7.138671e-04 3.569336e-03
3            control_w_explain - interrupt -3.6419265 2.706053e-04 1.623632e-03
4            control - interrupt_w_explain -1.5720916 1.159293e-01 2.318586e-01
5  control_w_explain - interrupt_w_explain -1.8298209 6.727672e-02 2.691069e-01
6          interrupt - interrupt_w_explain  1.8121056 6.996988e-02 2.099096e-01
7                   control - no_interrupt -7.2992825 2.893064e-13 2.603758e-12
8         control_w_explain - no_interrupt -7.5570118 4.124347e-14 4.124347e-13
9                 interrupt - no_interrupt -3.9150853 9.037226e-05 6.326058e-04
10      interrupt_w_explain - no_interrupt -5.7271909 1.021073e-08 8.168582e-08

### 4. Detection Number (averaged across both rounds)

In [15]:
kruskal.test(data$detection_num ~ factor(data$condition_n_explanation)) 
kruskal.test(data$detection_num ~ data$round)
kruskal.test(data$detection_num ~ data$condition_n_round)


	Kruskal-Wallis rank sum test

data:  data$detection_num by factor(data$condition_n_explanation)
Kruskal-Wallis chi-squared = 11.822, df = 4, p-value = 0.01873



	Kruskal-Wallis rank sum test

data:  data$detection_num by data$round
Kruskal-Wallis chi-squared = 0.0042673, df = 1, p-value = 0.9479



	Kruskal-Wallis rank sum test

data:  data$detection_num by data$condition_n_round
Kruskal-Wallis chi-squared = 12.623, df = 9, p-value = 0.1804


In [16]:
dunnTest(data$detection_num, factor(data$condition_n_explanation))

Dunn (1964) Kruskal-Wallis multiple comparison

  p-values adjusted with the Holm method.




                                Comparison           Z     P.unadj      P.adj
1              control - control_w_explain -0.04442458 0.964565966 0.96456597
2                      control - interrupt -2.18513422 0.028879007 0.23103205
3            control_w_explain - interrupt -2.14070964 0.032297461 0.22608223
4            control - interrupt_w_explain -1.71923140 0.085572246 0.51343347
5  control_w_explain - interrupt_w_explain -1.67480681 0.093972142 0.46986071
6          interrupt - interrupt_w_explain  0.46590282 0.641285060 1.00000000
7                   control - no_interrupt -2.59828285 0.009369128 0.09369128
8         control_w_explain - no_interrupt -2.55385827 0.010653661 0.09588295
9                 interrupt - no_interrupt -0.41314863 0.679497722 1.00000000
10      interrupt_w_explain - no_interrupt -0.87905145 0.379373378 1.00000000

### 5. Interruption Number

In [17]:
kruskal.test(data$slippery_interruption_num_fog ~ factor(data$condition_n_explanation)) 
kruskal.test(data$interruption_num_fog ~ data$extra_step)
kruskal.test(data$slippery_interruption_num_fog ~ data$condition_n_round)
kruskal.test(data$slippery_interruption_num_non_fog ~ factor(data$condition_n_explanation)) 
kruskal.test(data$slippery_interruption_num_non_fog ~ data$round)
kruskal.test(data$slippery_interruption_num_non_fog ~ data$condition_n_round)


	Kruskal-Wallis rank sum test

data:  data$slippery_interruption_num_fog by factor(data$condition_n_explanation)
Kruskal-Wallis chi-squared = 122.46, df = 4, p-value < 2.2e-16



	Kruskal-Wallis rank sum test

data:  data$interruption_num_fog by data$extra_step
Kruskal-Wallis chi-squared = 229.53, df = 180, p-value = 0.007392



	Kruskal-Wallis rank sum test

data:  data$slippery_interruption_num_fog by data$condition_n_round
Kruskal-Wallis chi-squared = 130, df = 9, p-value < 2.2e-16



	Kruskal-Wallis rank sum test

data:  data$slippery_interruption_num_non_fog by factor(data$condition_n_explanation)
Kruskal-Wallis chi-squared = 125.32, df = 4, p-value < 2.2e-16



	Kruskal-Wallis rank sum test

data:  data$slippery_interruption_num_non_fog by data$round
Kruskal-Wallis chi-squared = 0.84537, df = 1, p-value = 0.3579



	Kruskal-Wallis rank sum test

data:  data$slippery_interruption_num_non_fog by data$condition_n_round
Kruskal-Wallis chi-squared = 127.4, df = 9, p-value < 2.2e-16


## Subjective Metrics

### 1. User Preference

In [18]:
rank_data <- read.csv("rank_data_user_study_1.csv")

In [19]:
kruskal.test(rank_data$rank ~ factor(rank_data$condition_n_explanation)) 


	Kruskal-Wallis rank sum test

data:  rank_data$rank by factor(rank_data$condition_n_explanation)
Kruskal-Wallis chi-squared = 61.675, df = 4, p-value = 1.289e-12


In [20]:
dunnTest(rank_data$rank, factor(rank_data$condition_n_explanation))

Dunn (1964) Kruskal-Wallis multiple comparison

  p-values adjusted with the Holm method.




                                Comparison          Z      P.unadj        P.adj
1              control - control_w_explain -4.1637128 3.131139e-05 2.504912e-04
2                      control - interrupt -3.3309702 8.654386e-04 5.192631e-03
3            control_w_explain - interrupt  0.8327426 4.049899e-01 4.049899e-01
4            control - interrupt_w_explain -7.4021561 1.339909e-13 1.339909e-12
5  control_w_explain - interrupt_w_explain -3.2384433 1.201839e-03 6.009196e-03
6          interrupt - interrupt_w_explain -4.0711858 4.677441e-05 3.274209e-04
7                   control - no_interrupt -1.7580121 7.874546e-02 2.362364e-01
8         control_w_explain - no_interrupt  2.4057007 1.614148e-02 6.456593e-02
9                 interrupt - no_interrupt  1.5729582 1.157285e-01 2.314570e-01
10      interrupt_w_explain - no_interrupt  5.6441440 1.660051e-08 1.494046e-07

In [21]:
Hadj <- unname(kruskal.test(rank_data$rank ~ factor(rank_data$condition_n_explanation))$statistic)
n <- sum(table(rank_data$rank, factor(rank_data$condition_n_explanation)))
e2 <- Hadj*(n+1)/(n^2-1)
print(Hadj)  # H-value (test statistic => chi-squared)
print(e2)  # Effect Size

[1] 61.67515
[1] 0.4282996


### 2. Trust

In [22]:
m <- lmer(data$trust ~ factor(data$condition_n_explanation) + factor(data$round)
          + factor(data$gender) + data$age + data$experience
          + data$extraversion + data$agreeableness + data$conscientiousness + data$neuroticism + data$intellect 
          + data$detection_num + data$reject_num + data$falling_num + data$total_interruption_num + (1|data$Id), data) 
shapiro.test(residuals(m))
print(leveneTest(data$trust ~ factor(data$condition_n_explanation) * factor(data$round) * factor(data$gender)))
print.anova_stats(m, data, 0.05)


	Shapiro-Wilk normality test

data:  residuals(m)
W = 0.99049, p-value = 0.05666


Levene's Test for Homogeneity of Variance (center = median)
       Df F value Pr(>F)
group  19  0.4518 0.9779
      270               
Analysis of Variance Table
                                     npar  Sum Sq Mean Sq F value
factor(data$condition_n_explanation)    4 201.531  50.383  4.6167
factor(data$round)                      1   1.521   1.521  0.1393
factor(data$gender)                     1  19.107  19.107  1.7508
data$age                                1  25.355  25.355  2.3234
data$experience                         1  11.642  11.642  1.0668
data$extraversion                       1  66.396  66.396  6.0841
data$agreeableness                      1  47.869  47.869  4.3863
data$conscientiousness                  1  27.075  27.075  2.4809
data$neuroticism                        1   6.567   6.567  0.6018
data$intellect                          1  19.350  19.350  1.7731
data$detection_num                      1  63.473  63.473  5.8162
data$reject_num                         1  18.

factor(data$condition_n_explanation):  F(4,290)=4.61671758811399     p = 0.00125804347404986

data$extraversion:  F(1,290)=6.08409200862896     p = 0.0142206321690392

data$agreeableness:  F(1,290)=4.38633305641969     p = 0.0370989562418996

data$detection_num:  F(1,290)=5.81618861979635     p = 0.016502754734081



In [23]:
summary(glht(m, linfct=mcp("factor(data$condition_n_explanation)"="Tukey")))


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: lmer(formula = data$trust ~ factor(data$condition_n_explanation) + 
    factor(data$round) + factor(data$gender) + data$age + data$experience + 
    data$extraversion + data$agreeableness + data$conscientiousness + 
    data$neuroticism + data$intellect + data$detection_num + 
    data$reject_num + data$falling_num + data$total_interruption_num + 
    (1 | data$Id), data = data)

Linear Hypotheses:
                                             Estimate Std. Error z value
control_w_explain - control == 0              0.27549    0.61436   0.448
interrupt - control == 0                      0.78461    0.64912   1.209
interrupt_w_explain - control == 0            1.58127    0.64789   2.441
no_interrupt - control == 0                  -0.01384    1.01340  -0.014
interrupt - control_w_explain == 0            0.50913    0.64721   0.787
interrupt_w_explain - control_w_explain == 0  1.30578

### 3. Willingness

In [24]:
m <- lmer(data$willingness ~ factor(data$condition_n_explanation) + factor(data$round)
          + factor(data$gender) + data$age + data$experience
          + data$extraversion + data$agreeableness + data$conscientiousness + data$neuroticism + data$intellect 
          + data$detection_num + data$reject_num + data$falling_num + data$total_interruption_num + (1|data$Id), data) 
shapiro.test(residuals(m))
print(leveneTest(data$willingness ~ factor(data$condition_n_explanation) * factor(data$round) * factor(data$gender)))
print.anova_stats(m, data, 0.05)


	Shapiro-Wilk normality test

data:  residuals(m)
W = 0.99234, p-value = 0.1408


Levene's Test for Homogeneity of Variance (center = median)
       Df F value Pr(>F)
group  19  0.7013 0.8168
      270               
Analysis of Variance Table
                                     npar Sum Sq Mean Sq F value
factor(data$condition_n_explanation)    4 715.36 178.840 24.8231
factor(data$round)                      1   0.12   0.124  0.0172
factor(data$gender)                     1   0.10   0.101  0.0140
data$age                                1   0.82   0.816  0.1133
data$experience                         1   9.91   9.909  1.3753
data$extraversion                       1   3.35   3.348  0.4647
data$agreeableness                      1   9.60   9.600  1.3324
data$conscientiousness                  1   4.73   4.730  0.6565
data$neuroticism                        1   4.99   4.989  0.6925
data$intellect                          1   3.30   3.304  0.4586
data$detection_num                      1  14.57  14.568  2.0221
data$reject_num                         1  11.44  11.437  

factor(data$condition_n_explanation):  F(4,290)=24.8230750475945     p = 1.17601967520786e-17



In [25]:
summary(glht(m, linfct=mcp("factor(data$condition_n_explanation)"="Tukey")))


	 Simultaneous Tests for General Linear Hypotheses

Multiple Comparisons of Means: Tukey Contrasts


Fit: lmer(formula = data$willingness ~ factor(data$condition_n_explanation) + 
    factor(data$round) + factor(data$gender) + data$age + data$experience + 
    data$extraversion + data$agreeableness + data$conscientiousness + 
    data$neuroticism + data$intellect + data$detection_num + 
    data$reject_num + data$falling_num + data$total_interruption_num + 
    (1 | data$Id), data = data)

Linear Hypotheses:
                                             Estimate Std. Error z value
control_w_explain - control == 0               1.2670     0.4992   2.538
interrupt - control == 0                       0.9411     0.5298   1.777
interrupt_w_explain - control == 0             2.6791     0.5284   5.070
no_interrupt - control == 0                   -2.3064     0.8307  -2.776
interrupt - control_w_explain == 0            -0.3259     0.5279  -0.617
interrupt_w_explain - control_w_explain == 0   

### 4. Robot Likeability

In [26]:
# Fails normality and homoscedasticity assumptions
m <- lmer(data$likeability ~ factor(data$condition_n_explanation) + factor(data$round) 
          + factor(data$gender) + data$age + data$experience
          + data$extraversion + data$agreeableness + data$conscientiousness + data$neuroticism + data$intellect 
          + data$detection_num + data$reject_num + data$falling_num + data$total_interruption_num + (1|data$Id), data)
shapiro.test(residuals(m))
print(leveneTest(data$likeability ~ factor(data$condition_n_explanation) * factor(data$round) * factor(data$gender)))
print.anova_stats(m, data, 0.05)


	Shapiro-Wilk normality test

data:  residuals(m)
W = 0.98968, p-value = 0.03781


Levene's Test for Homogeneity of Variance (center = median)
       Df F value  Pr(>F)  
group  19   1.628 0.04941 *
      270                  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Analysis of Variance Table
                                     npar Sum Sq Mean Sq F value
factor(data$condition_n_explanation)    4 397.38  99.345 11.0394
factor(data$round)                      1   6.08   6.083  0.6759
factor(data$gender)                     1   9.13   9.131  1.0147
data$age                                1   3.16   3.162  0.3514
data$experience                         1   0.02   0.016  0.0018
data$extraversion                       1  41.50  41.502  4.6118
data$agreeableness                      1   3.49   3.493  0.3881
data$conscientiousness                  1   1.92   1.919  0.2133
data$neuroticism                        1   0.05   0.053  0.0059
data$intellect                          1   1.16   1.164  0.1294
data$detection_num                      1  17.9

factor(data$condition_n_explanation):  F(4,290)=11.0394327906997     p = 2.43740164420129e-08

data$extraversion:  F(1,290)=4.6117794161739     p = 0.0325846359955802

data$falling_num:  F(1,290)=5.28231221322819     p = 0.0222569866282495



In [27]:
kruskal.test(data$likeability ~ factor(data$condition_n_explanation)) 


	Kruskal-Wallis rank sum test

data:  data$likeability by factor(data$condition_n_explanation)
Kruskal-Wallis chi-squared = 28.58, df = 4, p-value = 9.515e-06


In [28]:
dunnTest(data$likeability, factor(data$condition_n_explanation))

Dunn (1964) Kruskal-Wallis multiple comparison

  p-values adjusted with the Holm method.




                                Comparison          Z      P.unadj        P.adj
1              control - control_w_explain -1.6518791 9.855920e-02 3.942368e-01
2                      control - interrupt -0.7466560 4.552712e-01 4.552712e-01
3            control_w_explain - interrupt  0.9052231 3.653472e-01 7.306944e-01
4            control - interrupt_w_explain -3.7616553 1.687925e-04 1.519133e-03
5  control_w_explain - interrupt_w_explain -2.1097762 3.487764e-02 2.092658e-01
6          interrupt - interrupt_w_explain -3.0149993 2.569798e-03 2.055839e-02
7                   control - no_interrupt  1.2752128 2.022339e-01 6.067018e-01
8         control_w_explain - no_interrupt  2.9270920 3.421477e-03 2.395034e-02
9                 interrupt - no_interrupt  2.0218689 4.318990e-02 2.159495e-01
10      interrupt_w_explain - no_interrupt  5.0368682 4.732104e-07 4.732104e-06

In [29]:
Hadj <- unname(kruskal.test(data$likeability ~ factor(data$condition_n_explanation))$statistic)
n <- sum(table(data$likeability, factor(data$condition_n_explanation)))
e2 <- Hadj*(n+1)/(n^2-1)
print(Hadj)  # H-value (test statistic => chi-squared)
print(e2)  # Effect Size

[1] 28.57963
[1] 0.09889147
